asdasd




In [1]:
import itertools
import os
import pickle
import pandas as pd
from deap import creator
import numpy as np
import matplotlib.pyplot as plt

In [9]:


def parse_file(path):
    data = []
    with open(path, 'r') as f:
        lines = f.readlines()
        opt = int(lines[0].replace('optimum=', ''))
        time = float(lines[1].split(',')[0].replace('execution_time=', ''))
        trace = []
        for idx,f in enumerate(lines[3:]):
            parts = f.split(',')
            fitness = int(float(parts[0].replace('fitness=', '')))
            iteration = int(float(parts[1].replace('iteration=', '')))
            gap = float(parts[2].replace('gap=', ''))
            delta = int(float(parts[3].replace('delta=', '')))
            timestamp = float(parts[4].replace('timestamp=', ''))
            data.append((idx, opt, time, fitness, iteration, gap, delta, timestamp))
    return data


def parse_file_new(path):
    data = []
    with open(path, 'r') as f:
        lines = f.readlines()
        opt = int(lines[0].replace('optimum=', ''))
        time = float(lines[1].split(',')[0].replace('execution_time=', ''))
        overall_iterations = int(lines[2].split(',')[1].replace('overall_iterations=',''))
        trace = []
        for idx,f in enumerate(lines[4:]):
            parts = f.split(',')
            fitness = int(float(parts[0].replace('fitness=', '')))
            iteration = int(float(parts[1].replace('iteration=', '')))
            gap = float(parts[2].replace('gap=', ''))
            delta = int(float(parts[3].replace('delta=', '')))
            timestamp = float(parts[4].replace('timestamp=', ''))
            data.append((idx, opt, time, fitness, iteration, overall_iterations, gap, delta, timestamp))
    return data

In [6]:
def load_full_experiment(path, algs, ns, runs, ls):
    data = []
    for alg, n in list(itertools.product(algs, ns)):
        for i in runs:#result_100_aco_False_0
            # filename = path + "result_{}_{}_{}_{}.txt".format(alg, n, i, ls)
            filename = path + "result_{}_{}_{}_{}.txt".format(n,alg, ls, i)
            dat = parse_file_new(filename)
            data = data + [(alg,n,i)+d for d in dat]

    return data


def load_ga_aco_dnala(path, ls):
    algs = ["ga", "aco", "dnala"]
    ns = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]
    basepath = path
    runs = range(0, 5)
    data = load_full_experiment(basepath, algs, ns, runs, ls)

    df = pd.DataFrame(data=data, columns=['alg','n','i','idx','optimum', 'time', 'fitness', 'iteration', 'overall_iteration', 'gap', 'delta', 'timestamp'])
    df = df.set_index(['alg','n','i','idx'])

    df = df.groupby(by=['alg','n','i']).apply(lambda x: x.sort_values(['idx'], ascending=True)).groupby(by=['alg','n','i']).tail(1).groupby(by=['alg','n']).median()

    df['time'] = np.minimum(np.floor(df['time']),300)
    df = df.drop(columns=['timestamp','delta'])
    return df

print('NO-LocalSearch')
# without_ls = load_ga_aco_dnala('./output/results_nols/', False)
without_ls = load_ga_aco_dnala('./output/main-table-comparison/', False)

#print('WITH-LocalSearch 100BF')
#load_ga_aco_dnala('./output/results_100-ls/', True)

print('WITH-LocalSearch 50FF')
# with_ls = load_ga_aco_dnala('./output/results_50-ff-ls/', True)
with_ls = load_ga_aco_dnala('./output/main-table-comparison/', True)

with_ls.drop(columns='optimum',inplace=True)

without_ls = without_ls[['optimum','fitness','gap','time','iteration','overall_iteration']]
with_ls = with_ls[['fitness','gap','time','iteration','overall_iteration']]

main_table = pd.concat([without_ls, with_ls], axis=1)

print(main_table.to_latex(float_format="%.2f"))

#main_table.drop(columns=['optimum'], inplace=True)
#print(main_table)


NO-LocalSearch
WITH-LocalSearch 50FF
\begin{tabular}{llrrrrrrrrrrr}
\toprule
   &     &  optimum &  fitness &    gap &   time &  iteration &  overall\_iteration &  fitness &   gap &   time &  iteration &  overall\_iteration \\
alg & n &          &          &        &        &            &                    &          &       &        &            &                    \\
\midrule
aco & 10  &     1994 &     1994 &   0.00 &   0.00 &          0 &                  0 &     1994 &  0.00 &   0.00 &          0 &                  0 \\
   & 20  &    11758 &    11758 &   0.00 &   0.00 &          0 &                  0 &    11758 &  0.00 &   0.00 &          0 &                  0 \\
   & 30  &    19032 &    19032 &   0.00 &   0.00 &          1 &                  1 &    19032 &  0.00 &   0.00 &          0 &                  0 \\
   & 40  &    20892 &    20892 &   0.00 &   8.00 &         89 &                 89 &    20892 &  0.00 &   0.00 &          0 &                  0 \\
   & 50  &    34198 &   

In [4]:
def load_ga_population_comparison_full_experiment(path, ns, runs, population_sizes,alg):
    data = []
    for population_size, n in list(itertools.product(population_sizes, ns)):
        for i in runs:
            filename = path + "result_{}_{}_{}_{}.txt".format(n,alg,population_size, i)
            if not os.path.exists(filename):
                continue
            dat = parse_file(filename)
            data = data + [(n,i,population_size)+d for d in dat]

    return data

In [5]:
def load_population_comparison(path,instance_size,instance,convergence,alg):
    ns = [50,100]
    population_sizes = [20,100,300, 500]
    basepath = path
    runs = range(0, 4)
    data = load_ga_population_comparison_full_experiment(basepath, ns, runs, population_sizes,alg)

    raw = pd.DataFrame(data=data, columns=['n','i','population_size','idx','optimum', 'time', 'fitness', 'iteration', 'gap', 'delta', 'timestamp'])

    df = raw.set_index(['n','i','population_size','idx'])

    df = df.groupby(by=['population_size','n','i']).apply(lambda x: x.sort_values(['idx'], ascending=True)).groupby(by=['population_size','n','i']).tail(1).groupby(by=['population_size','n']).median()

    df['time'] = np.minimum(np.floor(df['time']),300)
    df = df.drop(columns=['timestamp','delta'])

    pg = raw.drop(columns=['iteration', 'idx','delta','fitness','time'])

    # df2 = raw.groupby(by=['population_size','n','i'])


    pop_50 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.population_size == 20)][['gap','timestamp']]
    pop_100 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.population_size == 100)][['gap','timestamp']]
    pop_300 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.population_size == 500)][['gap','timestamp']]

    pop_50 = pd.concat([pop_50, pd.DataFrame(data=[(pop_50['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    pop_100 = pd.concat([pop_100, pd.DataFrame(data=[(pop_100['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    pop_300 = pd.concat([pop_300, pd.DataFrame(data=[(pop_300['gap'].min(), convergence+100)], columns=['gap','timestamp'])])

    fig = plt.figure()
    fig.add_subplot(1,1,1)

    plt.plot(pop_50['timestamp'],pop_50['gap'], label='P=20', marker='.')
    plt.plot(pop_100['timestamp'],pop_100['gap'], label='P=100', marker='.')
    plt.plot(pop_300['timestamp'],pop_300['gap'], label='P=500', marker='.')
    plt.legend(loc='best')

    plt.xlabel('Execution Time [s]', fontsize=12)
    plt.ylabel('Optimality Gap [%]', fontsize=12)
    plt.xlim((0, convergence))

    plt.suptitle('Comparison population size, {}, n={}'.format(alg.upper(),instance_size), fontsize=14)

    plt.savefig('{}_{}_population_comparison.pdf'.format(alg,instance_size),bbox_inches='tight')

    # print(pop_50)
    # print(pop_100)
    # print(pop_300)

    return df


# load_ga_population_comparison('./output/ga_population_comparison/',instance_size = 100,instance = 1,convergence = 100)
load_population_comparison('./output/aco_population_comparison/',instance_size = 50, instance = 1,convergence = 200, alg='aco')
load_population_comparison('./output/aco_population_comparison/',instance_size = 100, instance = 1,convergence = 200, alg='aco')
load_population_comparison('./output/ga_population_comparison/',instance_size = 50, instance = 1,convergence = 200, alg='ga')
load_population_comparison('./output/ga_population_comparison/',instance_size = 100, instance = 1,convergence = 200, alg='ga')


ValueError: invalid literal for int() with base 10: ' 28)'

In [ ]:

def load_ga_mutation_rate_comparison_full_experiment(path, ns, runs, mr,alg):
    data = []
    for mutation_rate, n in list(itertools.product(mr, ns)):
        for i in runs:
            filename = path + "result_{}_{}_{}_{}.txt".format(n,alg,mutation_rate, i)
            if not os.path.exists(filename):
                continue
            dat = parse_file(filename)
            data = data + [(n,i,mutation_rate)+d for d in dat]

    return data

def load_mutation_rate_comparison(path,instance_size,instance,convergence,alg):
    mr = [1,4,8]
    ns = [50,100]
    basepath = path
    runs = range(0, 4)
    data = load_ga_mutation_rate_comparison_full_experiment(basepath, ns, runs, mr,alg)

    raw = pd.DataFrame(data=data, columns=['n','i','mutation_rate','idx','optimum', 'time', 'fitness', 'iteration', 'gap', 'delta', 'timestamp'])

    df = raw.set_index(['n','i','mutation_rate','idx'])

    df = df.groupby(by=['mutation_rate','n','i']).apply(lambda x: x.sort_values(['idx'], ascending=True)).groupby(by=['mutation_rate','n','i']).tail(1).groupby(by=['mutation_rate','n']).median()

    df['time'] = np.minimum(np.floor(df['time']),300)
    df = df.drop(columns=['timestamp','delta'])

    pg = raw.drop(columns=['iteration', 'idx','delta','fitness','time'])

    # df2 = raw.groupby(by=['population_size','n','i'])


    pop_50 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.mutation_rate == 1)][['gap','timestamp']]
    pop_100 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.mutation_rate == 4)][['gap','timestamp']]
    pop_300 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.mutation_rate == 8)][['gap','timestamp']]

    pop_50 = pd.concat([pop_50, pd.DataFrame(data=[(pop_50['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    pop_100 = pd.concat([pop_100, pd.DataFrame(data=[(pop_100['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    pop_300 = pd.concat([pop_300, pd.DataFrame(data=[(pop_300['gap'].min(), convergence+100)], columns=['gap','timestamp'])])

    fig = plt.figure()
    fig.add_subplot(1,1,1)

    plt.plot(pop_50['timestamp'],pop_50['gap'], label='r=1', marker='.')
    plt.plot(pop_100['timestamp'],pop_100['gap'], label='r=4', marker='.')
    plt.plot(pop_300['timestamp'],pop_300['gap'], label='r=8', marker='.')
    plt.legend(loc='best')

    plt.xlabel('Execution Time [s]', fontsize=12)
    plt.ylabel('Optimality Gap [%]', fontsize=12)
    plt.xlim((0, convergence))

    plt.suptitle('Comparison mutation rate, {}, n={}'.format(alg.upper(),instance_size), fontsize=14)

    plt.savefig('{}_{}_mutation_rate_comparison.pdf'.format(alg,instance_size),bbox_inches='tight')

    # print(pop_50)
    # print(pop_100)
    # print(pop_300)

    return df


load_mutation_rate_comparison('./output/ga_mutation_rate_comparison/', instance_size = 50, instance = 1,convergence = 100, alg='ga')


In [ ]:


def load_aco_params_comparison_full_experiment(path, ns, runs, alphas,betas,alg):
    data = []
    for alpha,beta, n in list(itertools.product(alphas, betas, ns)):
        for i in runs:
            filename = path + "result_{}_{}_{}_{}_{}.txt".format(n,alg,alpha,beta, i)
            if not os.path.exists(filename):
                continue
            dat = parse_file(filename)
            data = data + [(n,i,alpha,beta)+d for d in dat]

    return data

def load_aco_params_comparison(path,instance_size,instance,convergence,alg):
    alphas = [1,4,8]
    betas = [1,4,8]
    ns = [50,100]
    basepath = path
    runs = range(0, 4)
    data = load_aco_params_comparison_full_experiment(basepath, ns, runs, alphas,betas,alg)

    raw = pd.DataFrame(data=data, columns=['n','i','alpha','beta','idx','optimum', 'time', 'fitness', 'iteration', 'gap', 'delta', 'timestamp'])
    print(raw)
    df = raw.set_index(['n','i','alpha','beta','idx'])

    df = df.groupby(by=['alpha','beta','n','i']).apply(lambda x: x.sort_values(['idx'], ascending=True)).groupby(by=['alpha','beta','n','i']).tail(1).groupby(by=['alpha','beta','n']).median()

    df['time'] = np.minimum(np.floor(df['time']),300)
    df = df.drop(columns=['timestamp','delta'])

    pg = raw.drop(columns=['iteration', 'idx','delta','fitness','time'])

    # df2 = raw.groupby(by=['population_size','n','i'])


    p11 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.alpha == 1)& (pg.beta == 1)][['gap','timestamp']]
    p14 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.alpha == 1)& (pg.beta == 4)][['gap','timestamp']]
    p41 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.alpha == 4)& (pg.beta == 1)][['gap','timestamp']]
    p18 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.alpha == 1)& (pg.beta == 8)][['gap','timestamp']]
    p81 = pg[(pg.i == instance) & (pg.n == instance_size) & (pg.alpha == 8)& (pg.beta == 1)][['gap','timestamp']]

    p11 = pd.concat([p11, pd.DataFrame(data=[(p11['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    p14 = pd.concat([p14, pd.DataFrame(data=[(p14['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    p41 = pd.concat([p41, pd.DataFrame(data=[(p41['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    p18 = pd.concat([p18, pd.DataFrame(data=[(p18['gap'].min(), convergence+100)], columns=['gap','timestamp'])])
    p81 = pd.concat([p81, pd.DataFrame(data=[(p81['gap'].min(), convergence+100)], columns=['gap','timestamp'])])

    fig = plt.figure()
    fig.add_subplot(1,1,1)

    plt.plot(p11['timestamp'],p11['gap'], label='alpha=1, beta=1', marker='.')
    plt.plot(p14['timestamp'],p14['gap'], label='alpha=1, beta=4', marker='.')
    plt.plot(p41['timestamp'],p41['gap'], label='alpha=4, beta=1', marker='.')
    plt.plot(p18['timestamp'],p18['gap'], label='alpha=1, beta=8', marker='.')
    plt.plot(p81['timestamp'],p81['gap'], label='alpha=8, beta=1', marker='.')
    plt.legend(loc='best')

    plt.xlabel('Execution Time [s]', fontsize=12)
    plt.ylabel('Optimality Gap [%]', fontsize=12)
    plt.xlim((0, convergence))

    plt.suptitle('Comparison alpha/beta ratio, {}, n={}'.format(alg.upper(),instance_size), fontsize=14)

    plt.savefig('{}_{}_alpha_beta_ratio_comparison.pdf'.format(alg,instance_size),bbox_inches='tight')

    # print(pop_50)
    # print(pop_100)
    # print(pop_300)

    return df

load_aco_params_comparison('./output/aco_alpha_beta_comparison/', instance_size = 50, instance = 1,convergence = 100, alg='aco')

In [19]:
def load_full_experiment_mutation_type(path, ns, runs, types):
    data = []
    for type, n in list(itertools.product(types, ns)):
        for i in runs:#result_100_aco_False_0
            # filename = path + "result_{}_{}_{}_{}.txt".format(alg, n, i, ls)
            filename = path + "result_{}_{}_{}_{}_{}.txt".format(n,'ga', False,type, i)
            dat = parse_file_new(filename)
            data = data + [(n,i,type)+d for d in dat]

    return data


def load_mutation_type_comparison(path):
    types = [True,False]
    ns = [70,80,90,100,150,200]
    basepath = path
    runs = range(0, 4)
    data = load_full_experiment_mutation_type(basepath, ns, runs, types)

    raw = pd.DataFrame(data=data, columns=['n','i','type','idx','optimum', 'time', 'fitness', 'iteration','overall_iteration', 'gap', 'delta', 'timestamp'])

    df = raw.set_index(['n','i','type','idx'])

    df = df.groupby(by=['type','n','i']).apply(lambda x: x.sort_values(['idx'], ascending=True)).groupby(by=['type','n','i']).tail(1).groupby(by=['type','n']).median()

    df['time'] = np.minimum(np.floor(df['time']),300)
    df = df.drop(columns=['timestamp','delta'])
    df = df[['optimum','fitness','gap','time','iteration','overall_iteration']]
    # print(df)

    print(df.to_latex(float_format="%.2f"))
    # pg = raw.drop(columns=['iteration', 'idx','delta','fitness','time'])

    return df


load_mutation_type_comparison('./output/ga-improving-mutation/')

\begin{tabular}{llrrrrrr}
\toprule
      &     &   optimum &   fitness &    gap &   time &  iteration &  overall\_iteration \\
type & n &           &           &        &        &            &                    \\
\midrule
False & 70  &  47234.00 &  47371.00 &   0.29 & 300.00 &     278.50 &            2203.50 \\
      & 80  &  53538.00 &  55116.00 &   2.95 & 300.00 &     564.00 &            1650.00 \\
      & 90  &  69731.00 &  70490.00 &   1.09 & 300.00 &     599.00 &            1012.00 \\
      & 100 &  83495.00 &  94405.00 &  13.07 & 300.00 &     501.00 &             629.50 \\
      & 150 & 104772.00 & 180576.00 &  72.35 & 300.00 &     149.00 &             149.00 \\
      & 200 & 130178.00 & 280278.00 & 115.30 & 300.00 &     142.50 &             142.50 \\
True  & 70  &  47234.00 &  47366.00 &   0.28 & 221.00 &     320.00 &            1523.50 \\
      & 80  &  53538.00 &  53538.00 &   0.00 &  88.00 &     295.50 &             364.50 \\
      & 90  &  69731.00 &  71061.00 &   1.91 & 3

optimum   fitness         gap   time  iteration  overall_iteration
type  n                                                                       
False 70    47234.0   47371.0    0.290045  300.0      278.5             2203.5
      80    53538.0   55116.0    2.947439  300.0      564.0             1650.0
      90    69731.0   70490.0    1.088469  300.0      599.0             1012.0
      100   83495.0   94405.0   13.066651  300.0      501.0              629.5
      150  104772.0  180576.0   72.351392  300.0      149.0              149.0
      200  130178.0  280278.0  115.303661  300.0      142.5              142.5
True  70    47234.0   47366.0    0.279460  221.0      320.0             1523.5
      80    53538.0   53538.0    0.000000   88.0      295.5              364.5
      90    69731.0   71061.0    1.907330  300.0      486.0             1492.5
      100   83495.0   89561.0    7.265106  300.0      401.5             1002.5
      150  104772.0  192572.0   83.801016  300.0      135.5              135.5
      200  130178.0  279480.0  114.690654  300.0      143.5              143.5